In [6]:
import os

train_done = True
study_id = 'original'
data_id = 'pan'

size = '5p'
per = 30000

data_ = 'TransMut'
encoding_ = 'blosum'

id_ = 'OF_TData_pad14_{}_onlyBlosum_hlac_NOPOS'.format(size)

data_dir = '/home/s202357/thesis/transmut/data/transmut_github/'
model_folder = '/home/s202357/thesis/transmut/pipeline/{}/models/'.format(study_id)

result_folder = '/home/s202357/thesis/transmut/pipeline/{}/results/'.format(study_id)

plot_name = result_folder + id_+ '_plot_'
df_name = result_folder + id_+ '_df_'

if not os.path.exists(model_folder):
        print('creating dirs')
        os.makedirs(model_folder)
        os.makedirs(result_folder)
else:
        print('dirs exist')


dirs exist


In [7]:
!gpustat

mnemosyne            Tue Jun  7 16:24:31 2022  470.103.01
[0] NVIDIA TITAN V   | 40'C,  30 % |  2760 / 12066 MB | s202357(1379M) s202357(1377M)
[1] NVIDIA TITAN V   | 46'C,   8 % |  7602 / 12066 MB | s202357(3309M) s202357(4289M)
[2] NVIDIA TITAN V   | 39'C,   0 % |     0 / 12066 MB |
[3] NVIDIA TITAN V   | 61'C,  43 % |  3702 / 12066 MB | andab(1099M) andab(2599M)
[4] NVIDIA TITAN V   | 42'C,  10 % |  1617 / 12066 MB | ansup(1613M)
[5] NVIDIA TITAN V   | 33'C,   0 % |  3062 / 12066 MB | ansup(1617M) ansup(1441M)
[6] NVIDIA TITAN V   | 55'C,  35 % |  2603 / 12066 MB | andab(2599M)


In [8]:
os.environ['CUDA_VISIBLE_DEVICES']='2'

In [9]:
import math
from sklearn import metrics
from sklearn import preprocessing
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import time
import datetime
import random
random.seed(1234)

import warnings
warnings.filterwarnings("ignore")

from collections import Counter
from functools import reduce
from tqdm import tqdm, trange

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, auc, accuracy_score
#from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve, auc
#from sklearn.metrics import classification_report
#from sklearn.utils import class_weight

import sys
import gc
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data

#import framework as fr

In [10]:
sys.path.insert(0, '/home/s202357/thesis/transmut/source/')

import blosum as enc

In [11]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    use_cuda = True
    print('There are %d GPU(s) avalable.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

There are 1 GPU(s) avalable.


In [12]:
seed = 19961231
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [13]:
blosum_dict = {
'A' : [5,-2,-1,-2,-1,-1,-1,0,-2,-1,-2,-1,-1,-3,-1,1,0,-3,-2,0],
'R' : [-2,7,-1,-2,-4,1,0,-3,0,-4,-3,3,-2,-3,-3,-1,-1,-3,-1,-3],
'N' : [-1,-1,7,2,-2,0,0,0,1,-3,-4,0,-2,-4,-2,1,0,-4,-2,-3],
'D' : [-2,-2,2,8,-4,0,2,-1,-1,-4,-4,-1,-4,-5,-1,0,-1,-5,-3,-4],
'C' : [-1,-4,-2,-4,13,-3,-3,-3,-3,-2,-2,-3,-2,-2,-4,-1,-1,-5,-3,-1],
'Q' : [-1,1,0,0,-3,7,2,-2,1,-3,-2,2,0,-4,-1,0,-1,-1,-1,-3],
'E' : [-1,0,0,2,-3,2,6,-3,0,-4,-3,1,-2,-3,-1,-1,-1,-3,-2,-3],
'G' : [0,-3,0,-1,-3,-2,-3,8,-2,-4,-4,-2,-3,-4,-2,0,-2,-3,-3,-4],
'H' : [-2,0,1,-1,-3,1,0,-2,10,-4,-3,0,-1,-1,-2,-1,-2,-3,2,-4],
'I' : [-1,-4,-3,-4,-2,-3,-4,-4,-4,5,2,-3,2,0,-3,-3,-1,-3,-1,4],
'L' : [-2,-3,-4,-4,-2,-2,-3,-4,-3,2,5,-3,3,1,-4,-3,-1,-2,-1,1],
'K' : [-1,3,0,-1,-3,2,1,-2,0,-3,-3,6,-2,-4,-1,0,-1,-3,-2,-3],
'M' : [-1,-2,-2,-4,-2,0,-2,-3,-1,2,3,-2,7,0,-3,-2,-1,-1,0,1],
'F' : [-3,-3,-4,-5,-2,-4,-3,-4,-1,0,1,-4,0,8,-4,-3,-2,1,4,-1],
'P' : [-1,-3,-2,-1,-4,-1,-1,-2,-2,-3,-4,-1,-3,-4,10,-1,-1,-4,-3,-3],
'S' : [1,-1,1,0,-1,0,-1,0,-1,-3,-3,0,-2,-3,-1,5,2,-4,-2,-2],
'T' : [0,-1,0,-1,-1,-1,-1,-2,-2,-1,-1,-1,-1,-2,-1,2,5,-3,-2,0],
'W' : [-3,-3,-4,-5,-5,-1,-3,-3,-3,-3,-2,-3,-1,1,-4,-4,-3,15,2,-3],
'Y' : [-2,-1,-2,-3,-3,-1,-2,-3,2,-1,-1,-2,0,4,-3,-2,-2,2,8,-1],
'V' : [0,-3,-3,-4,-1,-3,-3,-4,-4,4,1,-3,1,-1,-3,-2,0,-3,-1,5]
}

for key, val in zip(blosum_dict.keys(), blosum_dict.values()):
    print(key, len(val))

A 20
R 20
N 20
D 20
C 20
Q 20
E 20
G 20
H 20
I 20
L 20
K 20
M 20
F 20
P 20
S 20
T 20
W 20
Y 20
V 20


In [14]:
def pep_to_blosum(pep):
    return [blosum_dict[word] for word in pep]

In [15]:
def best_treshold(df_results):
    acc_best = -999
    best_i = -999

    for i in np.arange(0, 1, 0.1):
        df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
        acc = accuracy_score(df_results['target'], df_results['pred_binary'])
        if acc > acc_best:
            df = df_results
            best_i = i
            acc_best = acc
            
    return acc_best, best_i, df


def best_treshold(df_results, t= []):
    acc_best = -999
    best_i = -999

    if t == []:
        for i in np.arange(0, 1, 0.1):
            df_results['pred_binary'] = np.where((df_results.pred > i) , 1, 0)
            acc = accuracy_score(df_results['target'], df_results['pred_binary'])
            if acc > acc_best:
                df = df_results
                best_i = i
                acc_best = acc
    else:
        best_i = t[0]
        df_results['pred_binary'] = np.where((df_results.pred > t[0]) , 1, 0)
        df = df_results
    return acc_best, best_i, df


def pkl(id_, n_layers, n_heads, model_num, d_model):
    return '{}_d{}_layer{}_multihead{}_MODEL{}.pkl'.format(id_, d_model, n_layers, n_heads, model_num)


def make_data_pep_hla(data):
    pep_inputs, hla_inputs, labels = [], [], []
    cc = 0
    tt = time.time()
    
    for pep, hla, label in zip(data.peptide, data.HLA_sequence, data.label):
        
        if cc % 100000 == 0:
            print("Blosum completed:", cc, end = " ")
            print(", Time:", round((time.time()-tt)/60, 2), "mins.")
            
        bl = pep_to_blosum(pep)
        max_len=14
        pep_len = len(pep)
        for t in range(max_len-pep_len):
            xs = [-1]*20
            bl.append(xs)
            
        pep_inputs.append(bl)
        bl_hla = pep_to_blosum(hla) 
        hla_inputs.append(bl_hla)

        cc += 1
    
    return torch.tensor(pep_inputs, dtype=torch.float).to(device), torch.tensor(hla_inputs, dtype=torch.float).to(device)


class MyDataSet(Data.Dataset):
    def __init__(self, pep_inputs, hla_inputs, labels):
        super(MyDataSet, self).__init__()
        self.pep_inputs = pep_inputs
        self.hla_inputs = hla_inputs
        self.labels = labels

    def __len__(self):
        l = self.pep_inputs.shape[0]
        return l

    def __getitem__(self, idx):
        try:
            p = self.pep_inputs[idx]
            h = self.hla_inputs[idx]
            lb = self.labels[idx]
        except:
            print("MyDataSet")
            print("shape pep_inputs", np.shape(self.pep_inputs))
            print("idx:", idx)
            #print("shape np.shape(pep_inputs[idx]) -> ", np.shape(self.pep_inputs[idx]))
            print(pep_inputs.shape[0])
            print(self.l)
        return p, h, lb

In [16]:
def get_attn_pad_mask(seq_q, seq_k):
    '''
    seq_q: [batch_size, seq_len]
    seq_k: [batch_size, seq_len]
    seq_len could be src_len or it could be tgt_len
    seq_len in seq_q and seq_len in seq_k maybe not equal
    '''
    batch_size, len_q, _ = seq_q.size()
    batch_size, len_k, _ = seq_k.size()
    
    seq_q = torch.empty(batch_size, len_q).to(device)
    seq_k = torch.empty(batch_size, len_k).to(device)
    
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size, 1, len_k], False is masked
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size, len_q, len_k]


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        '''
        Q: [batch_size, n_heads, len_q, d_k]
        K: [batch_size, n_heads, len_k, d_k]
        V: [batch_size, n_heads, len_v(=len_k), d_v]
        attn_mask: [batch_size, n_heads, seq_len, seq_len]
        '''
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k) # scores : [batch_size, n_heads, len_q, len_k]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is True.
        
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V) # [batch_size, n_heads, len_q, d_v]
        return context, attn


# In[17]:


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.W_Q = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_K = nn.Linear(d_model, d_k * n_heads, bias=False)
        self.W_V = nn.Linear(d_model, d_v * n_heads, bias=False)
        self.fc = nn.Linear(n_heads * d_v, d_model, bias=False)
    def forward(self, input_Q, input_K, input_V, attn_mask):
        '''
        input_Q: [batch_size, len_q, d_model]
        input_K: [batch_size, len_k, d_model]
        input_V: [batch_size, len_v(=len_k), d_model]
        attn_mask: [batch_size, seq_len, seq_len]
        '''
        
        residual, batch_size = input_Q, input_Q.size(0)

        Q = self.W_Q(input_Q).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # Q: [batch_size, n_heads, len_q, d_k]
        K = self.W_K(input_K).view(batch_size, -1, n_heads, d_k).transpose(1,2)  # K: [batch_size, n_heads, len_k, d_k]
        V = self.W_V(input_V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  # V: [batch_size, n_heads, len_v(=len_k), d_v]
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) # attn_mask : [batch_size, n_heads, seq_len, seq_len]
        
        context, attn = ScaledDotProductAttention()(Q, K, V, attn_mask)
        context = context.transpose(1, 2).reshape(batch_size, -1, n_heads * d_v) # context: [batch_size, len_q, n_heads * d_v]
        output = self.fc(context) # [batch_size, len_q, d_model]
        
        return nn.LayerNorm(d_model).to(device)(output + residual), attn


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.fc = nn.Sequential(
            nn.Linear(d_model, d_ff, bias=False),
            nn.ReLU(),
            nn.Linear(d_ff, d_model, bias=False)
        )
    def forward(self, inputs):
        '''
        inputs: [batch_size, seq_len, d_model]
        '''
        residual = inputs
        output = self.fc(inputs)
        return nn.LayerNorm(d_model).to(device)(output + residual) # [batch_size, seq_len, d_model]


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        # enc_outputs: [batch_size, src_len, d_model], attn: [batch_size, n_heads, src_len, src_len]
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size, src_len, d_model]
        return enc_outputs, attn


class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        #self.src_emb = nn.Embedding(vocab_size, d_model)
        #self.pos_emb = PositionalEncoding(d_model)
        #self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])

    def forward(self, enc_inputs):
        ##B## enc_outputs = self.src_emb(enc_inputs) # [batch_size, src_len, d_model]
        #enc_outputs = self.pos_emb(enc_inputs.transpose(0, 1)).transpose(0, 1) # [batch_size, src_len, d_model]
        enc_outputs = enc_inputs
        return enc_outputs, 0

    
class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        self.use_cuda = use_cuda
        device = torch.device('cuda')
        self.pep_encoder = Encoder().to(device)
        self.hla_encoder = Encoder().to(device)
        #self.decoder = Decoder().to(device)
        self.tgt_len = tgt_len
        self.projection = nn.Sequential(
                                        nn.Linear(tgt_len * d_model, 256),
                                        nn.ReLU(True),

                                        nn.BatchNorm1d(256),
                                        nn.Linear(256, 64),
                                        nn.ReLU(True),

                                        #output layer
                                        nn.Linear(64, 2)
                                        ).to(device)
        
    def forward(self, pep_inputs, hla_inputs):
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        '''
        # tensor to store decoder outputs
        # outputs = torch.zeros(batch_size, tgt_len, tgt_vocab_size).to(self.device)
        
        # enc_outputs: [batch_size, src_len, d_model], enc_self_attns: [n_layers, batch_size, n_heads, src_len, src_len]
        pep_enc_outputs, pep_enc_self_attns = self.pep_encoder(pep_inputs)
        hla_enc_outputs, hla_enc_self_attns = self.hla_encoder(hla_inputs)
        enc_outputs = torch.cat((pep_enc_outputs, hla_enc_outputs), 1) # concat pep & hla embedding
        #print("\n\nAfter concat:", enc_outputs, "\nsize", np.shape(enc_outputs))
        # dec_outpus: [batch_size, tgt_len, d_model], dec_self_attns: [n_layers, batch_size, n_heads, tgt_len, tgt_len], dec_enc_attn: [n_layers, batch_size, tgt_len, src_len]
        #dec_outputs, dec_self_attns = self.decoder(enc_outputs)
        dec_outputs = enc_outputs.view(enc_outputs.shape[0], -1) # Flatten [batch_size, tgt_len * d_model]
        #print("len(dec_outputs), len(dec_outputs[0])", len(dec_outputs), len(dec_outputs[0]))
        dec_logits = self.projection(dec_outputs) # dec_logits: [batch_size, tgt_len, tgt_vocab_size]

        return dec_logits.view(-1, dec_logits.size(-1)), 0, 0, 0

In [17]:
def performances(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    try:
        mcc = ((tp*tn) - (fn*fp)) / np.sqrt(np.float((tp+fn)*(tn+fp)*(tp+fp)*(tn+fn)))
    except:
        print('MCC Error: ', (tp+fn)*(tn+fp)*(tp+fp)*(tn+fn))
        mcc = np.nan
    sensitivity = tp/(tp+fn)
    specificity = tn/(tn+fp)
    
    try:
        recall = tp / (tp+fn)
    except:
        recall = np.nan
        
    try:
        precision = tp / (tp+fp)
    except:
        precision = np.nan
        
    try: 
        f1 = 2*precision*recall / (precision+recall)
    except:
        f1 = np.nan
        
    roc_auc = roc_auc_score(y_true, y_prob)
    prec, reca, _ = precision_recall_curve(y_true, y_prob)
    aupr = auc(reca, prec)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('auc={:.4f}|sensitivity={:.4f}|specificity={:.4f}|acc={:.4f}|mcc={:.4f}'.format(roc_auc, sensitivity, specificity, accuracy, mcc))
        print('precision={:.4f}|recall={:.4f}|f1={:.4f}|aupr={:.4f}'.format(precision, recall, f1, aupr))
    
    return (roc_auc, accuracy, mcc, f1, sensitivity, specificity, precision, recall, aupr)



def performances_QM_(y_true, y_pred, y_prob, print_ = True):
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels = [0, 1]).ravel().tolist()
    accuracy = (tp+tn)/(tn+fp+fn+tp)
    
    if print_:
        print('tn = {}, fp = {}, fn = {}, tp = {}'.format(tn, fp, fn, tp))
        print('y_pred: 0 = {} | 1 = {}'.format(Counter(y_pred)[0], Counter(y_pred)[1]))
        print('y_true: 0 = {} | 1 = {}'.format(Counter(y_true)[0], Counter(y_true)[1]))
        print('acc={:.4f}'.format(accuracy))
    
    return (fn, tp, accuracy)


def transfer(y_prob, threshold = 0.5):
    return np.array([[0, 1][x > threshold] for x in y_prob])


f_mean = lambda l: sum(l)/len(l)


def performances_to_pd(performances_list):
    metrics_name = ['roc_auc', 'accuracy', 'mcc', 'f1', 'sensitivity', 'specificity', 'precision', 'recall', 'aupr']

    performances_pd = pd.DataFrame(performances_list, columns = metrics_name)
    performances_pd.loc['mean'] = performances_pd.mean(axis = 0)
    performances_pd.loc['std'] = performances_pd.std(axis = 0)
    
    return performances_pd


In [18]:
def train_step(model, train_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device("cuda" if use_cuda else "cpu")
    
    time_train_ep = 0
    model.train()
    y_true_train_list, y_prob_train_list = [], []
    loss_train_list, dec_attns_train_list = [], []
    tr = 0
    for train_pep_inputs, train_hla_inputs, train_labels in train_loader:
        '''
        pep_inputs: [batch_size, pep_len]
        hla_inputs: [batch_size, hla_len]
        train_outputs: [batch_size, 2]
        '''
        tr += 1
        train_pep_inputs, train_hla_inputs, train_labels = train_pep_inputs.to(device), train_hla_inputs.to(device), train_labels.to(device)

        t1 = time.time()
        train_outputs, _, _, train_dec_self_attns = model(train_pep_inputs, train_hla_inputs)
        train_loss = criterion(train_outputs, train_labels)
        time_train_ep += time.time() - t1

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        y_true_train = train_labels.cpu().numpy()
        y_prob_train = nn.Softmax(dim = 1)(train_outputs)[:, 1].cpu().detach().numpy()
        
        y_true_train_list.extend(y_true_train)
        y_prob_train_list.extend(y_prob_train)
        loss_train_list.append(train_loss)
#         dec_attns_train_list.append(train_dec_self_attns)
        
    y_pred_train_list = transfer(y_prob_train_list, threshold)
    ys_train = (y_true_train_list, y_pred_train_list, y_prob_train_list)
    
    print('Fold-{}****Train (Ep avg): Epoch-{}/{} | Loss = {:.4f} | Time = {:.4f} sec'.format(fold, epoch, epochs, f_mean(loss_train_list), time_train_ep))
    metrics_train = performances(y_true_train_list, y_pred_train_list, y_prob_train_list, print_ = True)
    
    return ys_train, loss_train_list, metrics_train, time_train_ep#, dec_attns_train_list


def eval_step(model, val_loader, fold, epoch, epochs, use_cuda = True):
    device = torch.device('cuda')
    
    model.eval()
    torch.manual_seed(19961231)
    torch.cuda.manual_seed(19961231)
    with torch.no_grad():
        loss_val_list, dec_attns_val_list = [], []
        y_true_val_list, y_prob_val_list = [], []
        for val_pep_inputs, val_hla_inputs, val_labels in val_loader:
            val_pep_inputs, val_hla_inputs, val_labels = val_pep_inputs.to(device), val_hla_inputs.to(device), val_labels.to(device)
            val_outputs, _, _, val_dec_self_attns = model(val_pep_inputs, val_hla_inputs)
            val_loss = criterion(val_outputs, val_labels)

            y_true_val = val_labels.cpu().numpy()
            y_prob_val = nn.Softmax(dim = 1)(val_outputs)[:, 1].cpu().detach().numpy()

            y_true_val_list.extend(y_true_val)
            y_prob_val_list.extend(y_prob_val)
            loss_val_list.append(val_loss)
#             dec_attns_val_list.append(val_dec_self_attns)
            
        y_pred_val_list = transfer(y_prob_val_list, threshold)
        ys_val = (y_true_val_list, y_pred_val_list, y_prob_val_list)
        
        print('Fold-{} ****Test  Epoch-{}/{}: Loss = {:.6f}'.format(fold, epoch, epochs, f_mean(loss_val_list)))
        
        metrics_val = performances(y_true_val_list, y_pred_val_list, y_prob_val_list)
    return ys_val, loss_val_list, metrics_val, y_prob_val_list, y_pred_val_list #, dec_attns_val_list

In [19]:
pep_max_len = 14 # peptide; enc_input max sequence length
hla_max_len = 34 # hla; dec_input(=dec_output) max sequence length
tgt_len = pep_max_len + hla_max_len
pep_max_len, hla_max_len

vocab = np.load( data_dir + 'Transformer_vocab_dict.npy', allow_pickle = True).item()
vocab_size = len(vocab)

# Transformer Parameters
d_model = 20  # Embedding Size
d_k = d_v = 20  # dimension of K(=Q), V

d_ff = 512 # FeedForward dimension
n_layers = 1  # number of Encoder of Decoder Layer
n_heads = 3

batch_size = 1024
epochs = 25
threshold = 0.5

In [20]:
cv = 5
count = 0
index_order = []

for outer in range(cv):
        #print(f"\nk_outer: {outer}")
        #print("test: " + str(outer))
        test_idx = outer
        for inner in range(cv):
            #print(f"\t\nk_inner: {inner}")
            if inner!=outer:
                #print("\tval: " + str(inner))
                val_idx = inner
                train_index = list()
                for t in range(5):
                    if t!=inner and t!=outer:
                        train_index.append(t)
                #print("\ttrain:", str(train_index))
                count += 1  
                index_order.append([test_idx, val_idx, train_index])
                
index_order_nested = index_order


In [21]:
def data_with_loader(index_order_idx, type_ ='train', fold = None,  batch_size = 1024):
        
    if type_ == 'train':
        print("prepare training data:")
        df_list_pep = []
        df_list_hla = []
        df_list_labels = []
            
        idx_0 = index_order_nested[index_order_idx][2][0]
        idx_1 = index_order_nested[index_order_idx][2][1]
        idx_2 = index_order_nested[index_order_idx][2][2]
        
        for i in [idx_0, idx_1, idx_2]:
            data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(i), index_col = 0)[0:per]
            pep_inputs, hla_inputs = make_data_pep_hla(data)
            labels = torch.LongTensor( [x for x in data.label] ).to(device)
            df_list_pep.append(pep_inputs)
            df_list_hla.append(hla_inputs)
            df_list_labels.append(labels)     
        
        pep_inputs = torch.cat( (df_list_pep[0], df_list_pep[1], df_list_pep[2]) , 0)
        hla_inputs = torch.cat( (df_list_hla[0], df_list_hla[1], df_list_hla[2]) , 0)
        labels = torch.cat( (df_list_labels[0], df_list_labels[1], df_list_labels[2]) , 0)
        print("TRAIN ID:", [idx_0, idx_1, idx_2], len(pep_inputs))
        print("prepare training data done.")
        
    elif type_ == 'test':
        print("prepare test data:")
        idx = index_order_nested[index_order_idx][0]
        data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(idx), index_col = 0)[0:per]
        pep_inputs, hla_inputs = make_data_pep_hla(data)
        labels = torch.LongTensor( [x for x in data.label] ).to(device)
        print("TEST ID:", idx, len(pep_inputs))
        print("prepare test data done.")
        
    elif type_ == 'val':
        print("prepare val data:")
        idx = index_order_nested[index_order_idx][1]
        data = pd.read_csv(data_dir + 'upd_{}_d.csv'.format(idx), index_col = 0)[0:per]
        pep_inputs, hla_inputs = make_data_pep_hla(data)
        labels = torch.LongTensor( [x for x in data.label] ).to(device)
        print("VAL ID:", idx, len(pep_inputs))
        print("prepare val data done.")
        
    print("shape pep inputs:", np.shape(pep_inputs))
    print("shape hla inputs:", np.shape(hla_inputs))
    print("shape labels:", np.shape(labels))
    
    loader = Data.DataLoader(MyDataSet(pep_inputs, hla_inputs, labels), batch_size, shuffle = False, num_workers = 0)
    
    return pep_inputs, hla_inputs, labels, loader




In [23]:
fold_avg_list = [list(range(0,4)), list(range(4,8)), list(range(8,12)), list(range(12,16)), list(range(16,20))] 

In [24]:
ys_train_fold_dict, ys_val_fold_dict = {}, {}
train_fold_metrics_list, val_fold_metrics_list, test_fold_metrics_list = [], [], []
attns_train_fold_dict, attns_val_fold_dict = {}, {}
loss_train_fold_dict, loss_val_fold_dict, = {}, {}
metrics_all_val = [] 
metrics_all_train = []

time_train_c = time.time()
fold_avg_index = -1
for fold in tqdm(range(20)):
    
        test_bool = False
        #if fold % 4 == 0 and fold != 0:
        #    test_bool = True
        #    fold_avg_index += 1
            
        print('=====Inner Fold-{}====='.format(fold))
        print('-----Generate data loader-----')
        index_order_idx = fold
        print(index_order_idx)
        train_pep_inputs, train_hla_inputs, train_labels, train_loader = data_with_loader(index_order_idx, type_ = 'train', fold = fold,  batch_size = batch_size)
        val_pep_inputs, val_hla_inputs, val_labels, val_loader = data_with_loader(index_order_idx, type_ = 'val', fold = fold,  batch_size = batch_size)

        print('TEST ID', index_order_nested[index_order_idx][0])

        print('-----Compile model-----')
        model = Transformer()
        #model = nn.DataParallel(model)
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr = 1e-3) #, momentum = 0.99)

        print('-----Train-----')
        path_saver = model_folder + pkl(id_, n_layers, n_heads, fold, d_model)

        print('path_saver: ', path_saver)

        metric_best, ep_best = 0, -1
        time_train = 0

        for epoch in range(1, epochs + 1):
            print("Epoch:", epoch)
            time_train_f = time.time()
            ys_train, loss_train_list, metrics_train, time_train_ep = train_step(model, train_loader, fold, epoch, epochs, use_cuda) # , dec_attns_train
            ys_val, loss_val_list, metrics_val, _prob, _binary = eval_step(model, val_loader, fold, epoch, epochs, use_cuda) #, dec_attns_val

            train_fold_metrics_list.append(metrics_train)
            val_fold_metrics_list.append(metrics_val)

            metrics_ep_avg = sum(metrics_val[:4])/4
            if metrics_ep_avg > metric_best: 
                metric_best, ep_best = metrics_ep_avg, epoch
                #torch.save(model.eval().state_dict(), path_saver)
                torch.save(model.state_dict(), path_saver)
            print("Fold time: {:6.2f} mins.".format( round( (time.time()-time_train_f)/60, 2) ))
        print("best_epoch:", ep_best)
        print('-----Optimization Finished!-----')
        del train_pep_inputs, train_hla_inputs, train_labels, train_loader
        del val_pep_inputs, val_hla_inputs, val_labels, val_loader
        gc.collect()
        
print("Total training time: {:6.2f} mins.".format( round( (time.time()-time_train_c)/60, 2) ))


  0%|                                                                                                             | 0/20 [00:00<?, ?it/s]

=====Inner Fold-0=====
-----Generate data loader-----
0
prepare training data:
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
TRAIN ID: [2, 3, 4] 90000
prepare training data done.
shape pep inputs: torch.Size([90000, 14, 20])
shape hla inputs: torch.Size([90000, 34, 20])
shape labels: torch.Size([90000])
prepare val data:
Blosum completed: 0 , Time: 0.0 mins.
VAL ID: 1 30000
prepare val data done.
shape pep inputs: torch.Size([30000, 14, 20])
shape hla inputs: torch.Size([30000, 34, 20])
shape labels: torch.Size([30000])
TEST ID 0
-----Compile model-----
-----Train-----
path_saver:  /home/s202357/thesis/transmut/pipeline/original/models/OF_TData_pad14_5p_onlyBlosum_hlac_NOPOS_d20_layer1_multihead3_MODEL0.pkl
Epoch: 1
Fold-0****Train (Ep avg): Epoch-1/25 | Loss = 0.3903 | Time = 0.0604 sec
tn = 36281, fp = 8686, fn = 7052, tp = 37981
y_pred: 0 = 43333 | 1 = 46667
y_true: 0 = 44967 | 1 = 45033
auc=0.9068|sensitivity=0.843

Fold-0 ****Test  Epoch-13/25: Loss = 0.253111
tn = 13419, fp = 1638, fn = 1307, tp = 13636
y_pred: 0 = 14726 | 1 = 15274
y_true: 0 = 15057 | 1 = 14943
auc=0.9629|sensitivity=0.9125|specificity=0.8912|acc=0.9018|mcc=0.8039
precision=0.8928|recall=0.9125|f1=0.9025|aupr=0.9617
Fold time:   0.05 mins.
Epoch: 14
Fold-0****Train (Ep avg): Epoch-14/25 | Loss = 0.1856 | Time = 0.0647 sec
tn = 41638, fp = 3329, fn = 3165, tp = 41868
y_pred: 0 = 44803 | 1 = 45197
y_true: 0 = 44967 | 1 = 45033
auc=0.9788|sensitivity=0.9297|specificity=0.9260|acc=0.9278|mcc=0.8557
precision=0.9263|recall=0.9297|f1=0.9280|aupr=0.9786
Fold-0 ****Test  Epoch-14/25: Loss = 0.253411
tn = 13352, fp = 1705, fn = 1278, tp = 13665
y_pred: 0 = 14630 | 1 = 15370
y_true: 0 = 15057 | 1 = 14943
auc=0.9630|sensitivity=0.9145|specificity=0.8868|acc=0.9006|mcc=0.8015
precision=0.8891|recall=0.9145|f1=0.9016|aupr=0.9619
Fold time:   0.04 mins.
Epoch: 15
Fold-0****Train (Ep avg): Epoch-15/25 | Loss = 0.1821 | Time = 0.0620 sec
tn = 

  5%|█████                                                                                                | 1/20 [01:11<22:44, 71.82s/it]

Fold-0 ****Test  Epoch-25/25: Loss = 0.276322
tn = 13298, fp = 1759, fn = 1262, tp = 13681
y_pred: 0 = 14560 | 1 = 15440
y_true: 0 = 15057 | 1 = 14943
auc=0.9604|sensitivity=0.9155|specificity=0.8832|acc=0.8993|mcc=0.7991
precision=0.8861|recall=0.9155|f1=0.9006|aupr=0.9569
Fold time:   0.03 mins.
best_epoch: 21
-----Optimization Finished!-----
=====Inner Fold-1=====
-----Generate data loader-----
1
prepare training data:
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
TRAIN ID: [1, 3, 4] 90000
prepare training data done.
shape pep inputs: torch.Size([90000, 14, 20])
shape hla inputs: torch.Size([90000, 34, 20])
shape labels: torch.Size([90000])
prepare val data:
Blosum completed: 0 , Time: 0.0 mins.
VAL ID: 2 30000
prepare val data done.
shape pep inputs: torch.Size([30000, 14, 20])
shape hla inputs: torch.Size([30000, 34, 20])
shape labels: torch.Size([30000])
TEST ID 0
-----Compile model-----
-----Train-----
path_save

...

 95%|███████████████████████████████████████████████████████████████████████████████████████████████     | 19/20 [21:07<01:06, 66.58s/it]

Fold-18 ****Test  Epoch-25/25: Loss = 0.273447
tn = 13479, fp = 1593, fn = 1500, tp = 13428
y_pred: 0 = 14979 | 1 = 15021
y_true: 0 = 15072 | 1 = 14928
auc=0.9608|sensitivity=0.8995|specificity=0.8943|acc=0.8969|mcc=0.7938
precision=0.8939|recall=0.8995|f1=0.8967|aupr=0.9587
Fold time:   0.04 mins.
best_epoch: 20
-----Optimization Finished!-----
=====Inner Fold-19=====
-----Generate data loader-----
19
prepare training data:
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
Blosum completed: 0 , Time: 0.0 mins.
TRAIN ID: [0, 1, 2] 90000
prepare training data done.
shape pep inputs: torch.Size([90000, 14, 20])
shape hla inputs: torch.Size([90000, 34, 20])
shape labels: torch.Size([90000])
prepare val data:
Blosum completed: 0 , Time: 0.0 mins.
VAL ID: 3 30000
prepare val data done.
shape pep inputs: torch.Size([30000, 14, 20])
shape hla inputs: torch.Size([30000, 34, 20])
shape labels: torch.Size([30000])
TEST ID 4
-----Compile model-----
-----Train-----
path_s

Fold-19****Train (Ep avg): Epoch-13/25 | Loss = 0.1865 | Time = 0.0561 sec
tn = 41703, fp = 3377, fn = 3324, tp = 41596
y_pred: 0 = 45027 | 1 = 44973
y_true: 0 = 45080 | 1 = 44920
auc=0.9788|sensitivity=0.9260|specificity=0.9251|acc=0.9255|mcc=0.8511
precision=0.9249|recall=0.9260|f1=0.9255|aupr=0.9784
Fold-19 ****Test  Epoch-13/25: Loss = 0.251297
tn = 13774, fp = 1237, fn = 1760, tp = 13229
y_pred: 0 = 15534 | 1 = 14466
y_true: 0 = 15011 | 1 = 14989
auc=0.9634|sensitivity=0.8826|specificity=0.9176|acc=0.9001|mcc=0.8007
precision=0.9145|recall=0.8826|f1=0.8983|aupr=0.9615
Fold time:   0.04 mins.
Epoch: 14
Fold-19****Train (Ep avg): Epoch-14/25 | Loss = 0.1825 | Time = 0.0541 sec
tn = 41783, fp = 3297, fn = 3223, tp = 41697
y_pred: 0 = 45006 | 1 = 44994
y_true: 0 = 45080 | 1 = 44920
auc=0.9796|sensitivity=0.9283|specificity=0.9269|acc=0.9276|mcc=0.8551
precision=0.9267|recall=0.9283|f1=0.9275|aupr=0.9793
Fold-19 ****Test  Epoch-14/25: Loss = 0.260993
tn = 13919, fp = 1092, fn = 1983, t

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [22:12<00:00, 66.64s/it]

Fold-19 ****Test  Epoch-25/25: Loss = 0.280158
tn = 13841, fp = 1170, fn = 2044, tp = 12945
y_pred: 0 = 15885 | 1 = 14115
y_true: 0 = 15011 | 1 = 14989
auc=0.9599|sensitivity=0.8636|specificity=0.9221|acc=0.8929|mcc=0.7871
precision=0.9171|recall=0.8636|f1=0.8896|aupr=0.9591
Fold time:   0.04 mins.
best_epoch: 11
-----Optimization Finished!-----
Total training time:  22.21 mins.
